## This notebook is used for testing MTAOS communication with the AOS components: M1M3, M2, and hexapods

Step-by-step description of what we do can be found in 
Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T???? 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
aos = mtcs.rem.mtaos
mount = mtcs.rem.mtmount

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
await aos.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.ENABLED) #, settingsToApply='default') #leave this out!!

#### Check summary state of each CSC

In [ ]:
await checkAOSSummaryStates(aos, m1m3, m2, camhex, m2hex)

### Get M1M3 Ready: raise mirorr, turn on FB, clear forces

In [ ]:
print('Re-enabling M1M3')
await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

In [ ]:
#%matplotlib inline
await plotM1M3Forces(m1m3)

### Get M2 Ready: turn on FB, clear forces

In [ ]:
await salobj.set_summary_state(m2, salobj.State.STANDBY) 
#await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM2(m2)

In [ ]:
await plotM2Forces(m2)

### Get cam hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await readyHexForAOS(camhex)

In [ ]:
mount_angle = await mount.tel_elevation.next(flush=False, timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

### Get M2 hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await readyHexForAOS(m2hex)

### Step 1: add aberrations via OFC

In [ ]:
zernikes = np.zeros(19)
zerIdx = np.arange(4, 23)
zernikes[7-4] = 1 #add 1um of z7
print(zerIdx)
print(zernikes)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??
#await aos.cmd_issueCorrection.set_start() #in the future, we want to issue corrections manually

In [ ]:
ofc_dict =  await ofcCompSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

#### Look at telemetry to verify the changes in the DOFs

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.yForces0, label='M1M3 elevation y-force 101')
plt.plot(dfm2.lutGravity0, label='M2 elevation force B1')
plt.plot(dfh1.position1, label='Camera hexapod y')
plt.plot(dfh2.position1, label='M2 hexapod y')
plt.grid()
plt.legend()

### Step 2: add same aberrations twice. Are the aggregateDOF accumulating?

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
ofc_dict2 = await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot=True)

This depends on the control strategy we are using. For x00, do not expect 2 or 1.

In [ ]:
ofc_dict2['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
ofc_dict2['m2hexC'][:5]/ofc_dict2['m2hexC'][:5]

In [ ]:
plt.plot(ofc_dict2['aggregated_dof'][:10]/ofc_dict['aggregated_dof'][:10])

In [ ]:
plt.plot(ofc_dict2['visit_dof'][:10]/ofc_dict['visit_dof'][:10])

### Step 3: Reset the corrections

In [ ]:
await aos.cmd_resetCorrection.set_start()

In [ ]:
##the following should be done by cmd_resetCorrection, but right now this is not the case
await aos.cmd_issueCorrection.set_start() 
#issue the resetted correction, which should be equivalent to the below
#await m1m3.cmd_applyActiveOpticForces.set_start(zForces=[0]*156)
#await m2.cmd_resetForceOffsets.set_start()
#await camhex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)
#await m2hex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
ofc_dict3 =  await ofcCompSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

### Step 4: Double the aberration, do the corrections double? yes

In [ ]:
zernikes[7-4] = 2 # z7 = 2um

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
ofc_dict4 =  await ofcCompSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
ofc_dict4['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
ofc_dict4['m2hexC'][:5]/ofc_dict['m2hexC'][:5]

### Step 5: Observe the corrections being rejected

In [ ]:
await lowerM1M3(m1m3)

In [ ]:
#Expect this command to be rejected, because M1M3 is parked
zernikes[7-4] = 1
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

#### the addAberration command is supposed to have undone corrections to M2, camhex, and m2hex. Let's check that

In [ ]:
ofc_dict5 =  await ofcCompSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
print(ofc_dict5['m1m3C'] - ofc_dict4['m1m3C'])
print(ofc_dict5['m2C'] - ofc_dict4['m2C'])
print(ofc_dict5['camhexC'] - ofc_dict4['camhexC'])
print(ofc_dict5['m2hexC'] - ofc_dict4['m2hexC'])

#### What about corrections on the component side?

If the sent already agrees with applied in every case above (they should), checking the sent (=corrections) here is sufficient.

### Step 6: rejectCorrection 

This makes sure OFC does proper accounting when commands are rejected.

When cmd_addAberration doesn't issue corrections, we can test cmd_rejectCorrection. NOT NOW.

In [ ]:
await aos.cmd_rejectCorrection.set_start()

In [ ]:
ofc_dict6 =  await ofcCompSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)